# IMDB Random Forests Regression
![ImdbIcon](../images/imdbheader.jpg)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report

In [2]:
revenue_df = pd.read_csv('../data/revenue_df.csv')

In [3]:
revenue_df.columns

Index(['movie_title', 'year', 'tagline', 'plot', 'cast', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'avg_vote',
       'total_votes', 'us_voters_votes', 'votes', 'votes_1', 'votes_2',
       'votes_3', 'votes_4', 'votes_5', 'votes_6', 'votes_7', 'votes_8',
       'votes_9', 'votes_10', 'popularity', 'director_score', 'actor_score',
       'actress_score', 'tagline_sentiment', 'plot_sentiment', 'total_score',
       'profitable', 'budget', 'revenue', 'budget_adj', 'revenue_adj'],
      dtype='object')

## Random Forests Regression

In [4]:
features = ['duration', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 
            'Crime', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 
            'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'avg_vote',
            'total_votes', 'us_voters_votes', 'votes', 'votes_1', 'votes_2', 'votes_3', 'votes_4', 
            'votes_5', 'votes_6', 'votes_7', 'votes_8', 'votes_9', 'votes_10', 'popularity', 
            'director_score', 'actor_score', 'actress_score', 'tagline_sentiment', 'plot_sentiment', 
            'total_score', 'budget']

In [5]:
X = revenue_df[features]
y = revenue_df['revenue']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = 42)

In [7]:
rf = RandomForestRegressor()

In [8]:
print("Cross Val Score:", cross_val_score(rf, X_train, y_train, cv = 5).mean())

Cross Val Score: 0.6594561962148526


In [9]:
model_params = {}
count = 0

In [16]:
rf_params = {
    'max_depth': [9, 10, 11],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [3, 4, 5],
    'n_estimators': [300, 350, 400],
}

gs = GridSearchCV(rf, param_grid=rf_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient='index')

model_df

,max_depth,min_samples_leaf,min_samples_split,n_estimators,best_score,training_score,testing_score
model_1,3,1,2,200,0.631614,0.754532,0.470445
model_2,5,1,2,150,0.661995,0.850965,0.525663
model_3,6,1,3,150,0.667078,0.880465,0.565005
model_4,7,1,3,150,0.668255,0.901927,0.531659
model_5,8,1,4,200,0.674026,0.909740,0.529927
model_6,9,1,4,300,0.669716,0.921340,0.530033
model_7,11,1,3,350,0.671685,0.940367,0.533840


## AdaBoostRegressor

In [17]:
model_params_two = {}
count_two = 0

In [19]:
ada = AdaBoostRegressor(base_estimator=RandomForestRegressor())

ada_params = {
    'base_estimator__max_depth': [3, 4, 5],
    'learning_rate': [.85, .90, .95],
    'n_estimators': [150, 200, 250],
}

gs_two = GridSearchCV(ada, param_grid=ada_params, cv=3, n_jobs = 12)

gs_two.fit(X_train, y_train)

count_two += 1

gs_two.best_params_['best_score'] = gs_two.best_score_

gs_two.best_params_['training_score'] = gs_two.score(X_train, y_train)

gs_two.best_params_['testing_score'] = gs_two.score(X_test, y_test)

model_params_two[f'model_{count_two}'] = gs_two.best_params_

model_df_two = pd.DataFrame.from_dict(model_params_two, orient='index')

model_df_two

,base_estimator__max_depth,learning_rate,n_estimators,best_score,training_score,testing_score
model_1,3,0.90,150,0.621358,0.761560,0.379673
model_2,4,0.85,200,0.650423,0.839691,0.469446
